In [ ]:
import os
from os.path import join
import requests
import json
import urllib
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from slugify import slugify
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from pytube import YouTube

In [ ]:
song = """La policía te está extorsionando (dinero)
Pero ellos viven de lo que tú estás pagando
Y si te tratan como a un delincuente (ladrón)
No es tu culpa, dale gracias al regente

Hay que arrancar el problema de raíz
Y cambiar al gobierno de nuestro país
A la gente que está en la burocracia
A esa gente que le gustan las migajas

Yo por eso me quejo y me quejo
Porque aquí es donde vivo
Y yo ya no soy un pendejo

Que nos guachan los puestos del gobierno
Hay personas que se están enriqueciendo
Gente que vive en la pobreza
Nadie hace nada
Porque a nadie le interesa

Es la gente de arriba te detesta
Hay más gente que quiere
Que caigan sus cabezas
Si le das más poder al poder
Más duro te van a venir a coger"""

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
spanish_stopwords = set(list(punctuation)) 

In [ ]:
# Cleaning song
song_wo_stopwords = [word.lower() for word in word_tokenize(song) if word.lower() not in spanish_stopwords]
print(song_wo_stopwords)

In [ ]:
# Load words:
directories = ['amlo', 'presidencia', 'epn']
candidates = {}
for w in song_wo_stopwords:
    candidates[w] = []
for d in directories:
    directory = join('youtube-captions', d)
    for file in os.listdir(directory):
        if file.endswith("json"):
            with open(join(directory, file), 'r') as captions_file:
                video = json.load(captions_file)
                captions = video['captions_parsed']
                for caption in captions:
                    if caption['content'] == None:
                        continue
                    tokenized = word_tokenize(caption['content'])
                    for w1 in tokenized:
                        if w1.lower() in candidates:
                            caption['id'] = video['id']
                            caption['count'] = len(tokenized)
                            candidates[w1.lower()].append(caption)

In [ ]:
import math

video_url = "http://youtube.com/watch?v="
for candidate_key in candidates:
    print(candidate_key)
    if len(candidates[candidate_key]) > 0:
        candidates[candidate_key].sort(key=lambda x: x['count'], reverse=True)
        for text in candidates[candidate_key][:3]:
            seconds = float(text['start']) / 60
            ceil_seconds = math.floor(seconds)
            minutes = ceil_seconds
            seconds = round((seconds - ceil_seconds) * 60)
            print('\t' + str(text['count']), video_url +  text['id'] + "&t=" + str(minutes) +"m" + str(seconds)+ "s")